In [17]:
%cd twitteranalysis
import sqlite3
import environment


(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis


In [18]:
from SearchTools.WordMaps import get_adjacent_word_counts, get_adjacent_word_counts_in_tweets, get_user_ids_for_word, get_tweet_ids_for_word

# Properties of master.db

In [29]:

def _run_query(query, db=environment.MASTER_DB):
    conn = sqlite3.connect(db)
    with conn:
        r2 = conn.execute(query)
        amt = r2.fetchone()
        return amt[0]


def count_words(db=environment.MASTER_DB):
    query = "select count( distinct word) from word_map"
    amt = _run_query(query, db)
    print("%s distinct words in master.db" % amt)
    return amt

def count_rows(db=environment.MASTER_DB):
    query ="select count( *) from word_map"
    amt = _run_query(query, db)
    print("%s rows in %s" % (amt, db))
    return amt

def count_tweets(db=environment.MASTER_DB):
    query = "select count(distinct tweet_id) from word_map"
    amt = _run_query(query, db)
    print("%s tweets in master.db" % amt)
    return amt

# Distinct users
def count_users(db=environment.MASTER_DB):
    conn = sqlite3.connect(db)
    r = conn.execute("select count(distinct user_id) from word_map")
    n = r.fetchone()
    print("%s unique user ids in %s" % (n[0], db))
    conn.close()
    return n[0]


In [ ]:
def master_row_generator():
    conn = sqlite3.connect(environment.MASTER_DB)
    r = conn.execute("select * from word_map_deux")
    while True:
        yield r.fetchone()

def db_row_generator(filepath):
    conn = sqlite3.connect(filepath)
    r = conn.execute("select * from word_map_deux")
    while True:
        yield r.fetchone()


In [33]:
count_rows(environment.TWEET_DB_NO_STOP)

59064116 rows in /Users/adam/Dropbox/PainNarrativesLab/private_data/tweet-databases/tweets-no-stop.db


59064116

In [ ]:
%%time
count_words(environment.USER_DB_NO_STOP)
# environment.USER_DB_NO_STOP
# 11.4 without index
# 6.31 with index

In [ ]:
count_rows(environment.TWEET_DB_MASTER)

In [ ]:
count_tweets(environment.TWEET_DB_MASTER)

In [ ]:
count_users()

# Add indexes

In [7]:
import sqlite3
def add_indexes(db):
    fields = ['user_id', 'tweet_id', 'word']
    query = "create index %s_idx on word_map(%s)"
    conn = sqlite3.connect(db)
    with conn:
        for f in fields:
            q = query % (f, f)
            conn.execute(q)
    
# environment.TWEET_DB_MASTER
# 2.3GB before indexes added
# 5.88 GB after
    

In [8]:
# add_indexes(environment.TWEET_DB_NO_STOP)

# Explore data

In [38]:
# Distinct users
count_users()

0 unique user ids in /Users/adam/Desktop/TwitterDataAnalysisLogs/master.db


0

## row counts

In [ ]:
count_rows()

In [ ]:
count_words()

with exceptions in filter
    - 156590

without 
    - 164499

with regex
    - 144807, 145514

# Remove bad records

In [31]:
to_prune = [
    '\\n',
    '\\n\\n',
    '\\xc2\\x96',
    '\\xc3\\xa0',
    '\\xe2\\x80\\x93',
    '\\xe2\\x80\\x94',
    '\\xe2\\x80\\xa6',
    '\\xeb\\x8c\\x80\\xeb\\xb0\\x95',
    '\\xf0\\x9f\\x91\\x8a\\xf0\\x9f\\x8f\\xbb',
    '\\xf0\\x9f\\x98\\x94',
    '\\xf0\\x9f\\x98\\xa2',
    '\\xf0\\x9f\\x98\\xa9',
    '\\xf0\\x9f\\x98\\xab',
    '\\xf0\\x9f\\x98\\xad',
    '\\xf0\\x9f\\x98\\xbc\\xf0\\x9f\\x98\\xb9\\xf0\\x9f\\x99\\x80',
    '/\\xe2\\x80\\xa6',
    'h\\xe2\\x80\\xa6',
    'ht\\xe2\\x80\\xa6',
    'htt\\xe2\\x80\\xa6']

def prune_map(db):
    conn = sqlite3.connect(db)
    query = "DELETE FROM word_map WHERE word = ?"
    with conn:
        for t in manu_prune:
            word = (t, )
            conn.execute(query, word)
            print("deleted: %s" % t)

In [34]:
59756272 - 59064116

692156

In [32]:
prune_map(environment.TWEET_DB_NO_STOP)

# Before run: 
# 59756272 rows in /Users/adam/Dropbox/PainNarrativesLab/private_data/tweet-databases/tweets-no-stop.db
# After run:
# 59064116 rows in /Users/adam/Dropbox/PainNarrativesLab/private_data/tweet-databases/tweets-no-stop.db
# Removed 692,156 rows

deleted: \n
deleted: \n\n
deleted: \xc2\x96
deleted: \xc3\xa0
deleted: \xe2\x80\x93
deleted: \xe2\x80\x94
deleted: \xe2\x80\xa6
deleted: \xeb\x8c\x80\xeb\xb0\x95
deleted: \xf0\x9f\x91\x8a\xf0\x9f\x8f\xbb
deleted: \xf0\x9f\x98\x94
deleted: \xf0\x9f\x98\xa2
deleted: \xf0\x9f\x98\xa9
deleted: \xf0\x9f\x98\xab
deleted: \xf0\x9f\x98\xad
deleted: \xf0\x9f\x98\xbc\xf0\x9f\x98\xb9\xf0\x9f\x99\x80
deleted: /\xe2\x80\xa6
deleted: h\xe2\x80\xa6
deleted: ht\xe2\x80\xa6
deleted: htt\xe2\x80\xa6


## Word search

In [35]:
r = get_tweet_ids_for_word('pain')

In [36]:
len(set(r))

3026705

### Tweets

Pain
    
    Appears in 3,026,705 tweets

In [19]:
%%time

result = get_adjacent_word_counts_in_tweets('pain', offset=1, cutoff=500, db=environment.TWEET_DB_NO_STOP)

CPU times: user 8.73 s, sys: 1.23 s, total: 9.96 s
Wall time: 10 s


In [20]:
len(result)

260

In [22]:
result

[("'m", 969),
 ('--', 1273),
 ('..', 901),
 ('...', 24854),
 ('.buy', 679),
 ('/\\xe2\\x80\\xa6', 515),
 ('2nd', 687),
 ('8/11', 564),
 ('\\n', 5393),
 ('\\n\\n', 1481),
 ('\\xc2\\x96', 3232),
 ('\\xc3\\xa0', 653),
 ('\\xe2\\x80\\x93', 2657),
 ('\\xe2\\x80\\x94', 686),
 ('\\xe2\\x80\\xa6', 4007),
 ('\\xeb\\x8c\\x80\\xeb\\xb0\\x95', 20339),
 ('\\xf0\\x9f\\x91\\x8a\\xf0\\x9f\\x8f\\xbb', 993),
 ('\\xf0\\x9f\\x98\\x94', 531),
 ('\\xf0\\x9f\\x98\\xa2', 764),
 ('\\xf0\\x9f\\x98\\xa9', 1052),
 ('\\xf0\\x9f\\x98\\xab', 511),
 ('\\xf0\\x9f\\x98\\xad', 984),
 ('\\xf0\\x9f\\x98\\xbc\\xf0\\x9f\\x98\\xb9\\xf0\\x9f\\x99\\x80', 693),
 ('``', 2480),
 ('affects', 1472),
 ('always', 624),
 ('amp', 2161),
 ('anal', 536),
 ('anymore', 2905),
 ('anywhere', 1251),
 ('arthritis', 1168),
 ('assessment', 803),
 ('associated', 1164),
 ('au', 2353),
 ('aux', 2355),
 ('awareness', 2232),
 ('away', 2642),
 ('back', 1966),
 ('bad', 830),
 ('bakarkhani', 653),
 ('bc', 819),
 ('becomes', 582),
 ('behind', 866),
 ('be

### Tweets

Pain, offset -1; cutoff 500

    [("'m", 836),
     ('...', 2178),
     ('//\\xe2\\x80\\xa6', 535),
     ('//t.\\xe2\\x80\\xa6', 606),
     ('3d', 598),
     ('911well', 680),
     ('\\n', 665),
     ('\\xe2\\x80\\xa6', 694),
     ('``', 5405),
     ('abdominal', 875),
     ('acute', 755),
     ('always', 1233),
     ('american', 775),
     ('amp', 1667),
     ('animation', 1081),
     ('artesia', 2189),
     ('arthritis', 22385),
     ('back', 40339),
     ('backpain', 888),
     ('bad', 1695),
     ('best', 674),
     ('better', 575),
     ('biei', 1657),
     ('body', 1101),
     ('bone', 607),
     ('burning', 840),
     ('ca', 1429),
     ('cause', 1541),
     ('causes', 674),
     ('chest', 760),
     ('chronic', 62555),
     ('chronicpain', 3870),
     ('constant', 1697),
     ('could', 561),
     ('crps', 2346),
     ('cycle', 745),
     ('daily', 758),
     ('day', 1480),
     ('de', 770),
     ('diminish', 1079),
     ('doctors', 515),
     ('du', 779),
     ('ease', 1320),
     ('emotional', 1065),
     ('endometriosis', 1963),
     ('every', 711),
     ('excruciating', 915),
     ('experiencing', 600),
     ('extreme', 773),
     ('feel', 3182),
     ('feeling', 874),
     ('fibro', 2167),
     ('fibromyalgia', 8487),
     ('fight', 649),
     ('find', 699),
     ('first', 624),
     ('foot', 1864),
     ('free', 1443),
     ('freewebinar', 1274),
     ('fuck', 577),
     ('gardening', 713),
     ('get', 3135),
     ('getting', 707),
     ('go', 1048),
     ('going', 747),
     ('good', 1508),
     ('got', 2754),
     ('great', 1900),
     ('hand', 569),
     ('hate', 707),
     ('head', 635),
     ('headache', 767),
     ('health', 611),
     ('heightened', 827),
     ('help', 1768),
     ('high', 656),
     ('hip', 1221),
     ('home', 2016),
     ('hope', 639),
     ('hurt', 710),
     ('hurts', 718),
     ('injury', 590),
     ('intense', 969),
     ('joint', 11157),
     ('knee', 3798),
     ('know', 1911),
     ('labor', 4326),
     ('le', 1175),
     ('learn', 531),
     ('leg', 909),
     ('less', 2269),
     ('let', 945),
     ('life', 1143),
     ('like', 1601),
     ('limping', 1177),
     ('love', 1064),
     ('low', 933),
     ('lower', 1075),
     ('make', 757),
     ('makes', 556),
     ('manage', 969),
     ('managing', 567),
     ('mcgill', 514),
     ('migraine', 6364),
     ('much', 9450),
     ('muscle', 4204),
     ('muscular', 518),
     ('myofascial', 2311),
     ("n't", 2277),
     ('national', 3344),
     ('natural', 1369),
     ('neck', 5294),
     ('need', 1331),
     ('nerve', 6338),
     ('neuropathic', 2511),
     ('neuropathy', 4512),
     ('never', 994),
     ('new', 1602),
     ('night', 606),
     ('oh', 923),
     ('one', 1125),
     ('opioid', 555),
     ('ouch', 640),
     ('pain', 15397),
     ('pelvic', 2441),
     ('people', 548),
     ('period', 909),
     ('persistent', 599),
     ('physical', 2054),
     ('pick', 1695),
     ('please', 975),
     ('prescription', 1028),
     ('quit', 714),
     ('real', 841),
     ('really', 719),
     ('reduce', 2787),
     ('reduces', 961),
     ('regional', 7421),
     ('release', 587),
     ('relief', 1977),
     ('relieve', 1488),
     ('relieving', 956),
     ('remedies', 3108),
     ('remove', 842),
     ('right', 1217),
     ('rockfm', 504),
     ('ronitaylor', 528),
     ('rsd', 872),
     ('rt', 23562),
     ('sciatica', 1410),
     ('see', 546),
     ('seiring', 542),
     ('severe', 1656),
     ('shoulder', 2390),
     ('sleep', 681),
     ('someone', 806),
     ('something', 771),
     ('sometimes', 602),
     ('soul', 515),
     ('spine', 757),
     ('spoonie', 1278),
     ('stabbing', 597),
     ('still', 678),
     ('stomach', 504),
     ('stop', 644),
     ('take', 2732),
     ('tension', 606),
     ('terrible', 537),
     ('thanks', 698),
     ('think', 903),
     ('time', 1001),
     ('today', 1612),
     ('tooth', 634),
     ('topical', 651),
     ('tough', 706),
     ('treat', 1182),
     ('treating', 588),
     ('try', 640),
     ('turn', 1836),
     ('u', 527),
     ('ugh', 581),
     ('us', 1004),
     ('video', 660),
     ('w', 512),
     ('want', 789),
     ('well', 628),
     ('widespread', 2552),
     ('without', 1481),
     ('work', 658),
     ('worst', 1135),
     ('zero', 948)]

In [ ]:
### Tweets

Pain; offset +1; cutoff=500

    [("'m", 969),
     ('--', 1273),
     ('..', 901),
     ('...', 24854),
     ('.buy', 679),
     ('/\\xe2\\x80\\xa6', 515),
     ('2nd', 687),
     ('8/11', 564),
     ('\\n', 5393),
     ('\\n\\n', 1481),
     ('\\xc2\\x96', 3232),
     ('\\xc3\\xa0', 653),
     ('\\xe2\\x80\\x93', 2657),
     ('\\xe2\\x80\\x94', 686),
     ('\\xe2\\x80\\xa6', 4007),
     ('\\xeb\\x8c\\x80\\xeb\\xb0\\x95', 20339),
     ('\\xf0\\x9f\\x91\\x8a\\xf0\\x9f\\x8f\\xbb', 993),
     ('\\xf0\\x9f\\x98\\x94', 531),
     ('\\xf0\\x9f\\x98\\xa2', 764),
     ('\\xf0\\x9f\\x98\\xa9', 1052),
     ('\\xf0\\x9f\\x98\\xab', 511),
     ('\\xf0\\x9f\\x98\\xad', 984),
     ('\\xf0\\x9f\\x98\\xbc\\xf0\\x9f\\x98\\xb9\\xf0\\x9f\\x99\\x80', 693),
     ('``', 2480),
     ('affects', 1472),
     ('always', 624),
     ('amp', 2161),
     ('anal', 536),
     ('anymore', 2905),
     ('anywhere', 1251),
     ('arthritis', 1168),
     ('assessment', 803),
     ('associated', 1164),
     ('au', 2353),
     ('aux', 2355),
     ('awareness', 2232),
     ('away', 2642),
     ('back', 1966),
     ('bad', 830),
     ('bakarkhani', 653),
     ('bc', 819),
     ('becomes', 582),
     ('behind', 866),
     ('better', 1942),
     ('body', 915),
     ('ca', 1403),
     ('care', 2982),
     ('caused', 2648),
     ('causes', 947),
     ('changes', 4012),
     ('chanse_m', 744),
     ('chronic', 1135),
     ('chronicpain', 2691),
     ('clinic', 1736),
     ('clinics', 540),
     ('come', 720),
     ('comes', 1152),
     ('community', 687),
     ('condition', 1501),
     ('conditions', 1948),
     ('conference', 3360),
     ('control', 1819),
     ('cost', 508),
     ('could', 1529),
     ('cuit', 532),
     ('cure', 724),
     ('cures', 7386),
     ('dadpregnant', 1534),
     ('dat', 598),
     ('day', 3162),
     ('days', 1003),
     ('de', 3361),
     ('deception', 761),
     ('disease', 920),
     ('doc', 631),
     ('doctor', 780),
     ('drug', 512),
     ('due', 989),
     ('eat', 500),
     ('ebook', 577),
     ('email', 749),
     ('en', 1197),
     ('endometriosis', 568),
     ('est', 601),
     ('et', 970),
     ('even', 999),
     ('ever', 1258),
     ('every', 1436),
     ('feel', 1211),
     ('feels', 651),
     ('fibro', 685),
     ('fibromyalgia', 881),
     ('first', 1527),
     ('flare', 581),
     ('forever', 719),
     ('foundation', 3543),
     ('free', 6227),
     ('full', 802),
     ('get', 2318),
     ('gets', 637),
     ('getting', 780),
     ('girl', 502),
     ('girls', 1571),
     ('giving', 1267),
     ('go', 4223),
     ('god', 576),
     ('goes', 545),
     ('going', 855),
     ('gone', 1057),
     ('good', 926),
     ('got', 2269),
     ('gt', 1566),
     ('h\\xe2\\x80\\xa6', 839),
     ('health', 805),
     ('help', 1734),
     ('home', 760),
     ('hope', 597),
     ('ht\\xe2\\x80\\xa6', 1127),
     ('htt\\xe2\\x80\\xa6', 1468),
     ('hurt', 1153),
     ('hurts', 1429),
     ('indien', 833),
     ('intensity', 1102),
     ('issues', 732),
     ('killer', 1165),
     ('killers', 2297),
     ('killing', 4175),
     ('knee', 505),
     ('know', 948),
     ('la', 505),
     ('last', 542),
     ('lasts', 813),
     ('less', 753),
     ('let', 640),
     ('level', 1119),
     ('levels', 1923),
     ('life', 1079),
     ('like', 3084),
     ('linked', 828),
     ('lol', 920),
     ('love', 918),
     ('make', 959),
     ('makes', 2602),
     ('man', 578),
     ('management', 31714),
     ('matters', 4117),
     ('may', 4185),
     ('med', 607),
     ('medication', 3376),
     ('medications', 899),
     ('medicine', 2511),
     ('meds', 6420),
     ('mgmt', 880),
     ('migraine', 598),
     ('migraines', 1588),
     ('modeling', 529),
     ('much', 890),
     ('n', 2007),
     ("n't", 2976),
     ('na', 677),
     ('naturally', 3049),
     ('necklace', 588),
     ('need', 1265),
     ('never', 1209),
     ('new', 1128),
     ('news', 772),
     ('one', 1425),
     ('pain', 15397),
     ('patient', 1066),
     ('patients', 6704),
     ('people', 758),
     ('perception', 579),
     ('pills', 1123),
     ('playing', 667),
     ('please', 874),
     ('porn', 615),
     ('problems', 811),
     ('processing', 1574),
     ('push', 783),
     ('really', 1289),
     ('receptors', 3146),
     ('reduction', 778),
     ('related', 521),
     ('release', 1643),
     ('relief', 84066),
     ('relief4life', 8349),
     ('reliever', 2164),
     ('relievers', 1635),
     ('relieving', 1817),
     ('remedies', 2126),
     ('report', 1149),
     ('research', 2185),
     ('right', 2028),
     ('rt', 7898),
     ('ruin', 1894),
     ('scale', 2233),
     ('see', 774),
     ('sensitivity', 750),
     ('sex', 1277),
     ('side', 1852),
     ('sidelining', 1758),
     ('sign', 1879),
     ('signals', 1108),
     ('simulator', 1539),
     ('sleep', 670),
     ('slowing', 2544),
     ('slut', 2280),
     ('someone', 523),
     ('sometimes', 630),
     ('specialist', 1008),
     ('spoonie', 1145),
     ('still', 923),
     ('stop', 936),
     ('story', 515),
     ('strategy', 1076),
     ('sucks', 1058),
     ('suffer', 515),
     ('sufferers', 3077),
     ('summit', 614),
     ('surgery', 612),
     ('symptoms', 2002),
     ('syndrome', 9405),
     ('system', 924),
     ('take', 701),
     ('takes', 613),
     ('thanks', 767),
     ('therapy', 670),
     ('thetwittazine', 981),
     ('think', 635),
     ('threshold', 1241),
     ('time', 845),
     ('tip', 2584),
     ('tired', 1807),
     ('today', 5912),
     ('tolerance', 758),
     ('tonight', 828),
     ('trap', 795),
     ('treatment', 3178),
     ('treatments', 1036),
     ('try', 577),
     ('u', 1072),
     ('update', 1626),
     ('us', 3270),
     ('using', 606),
     ('usually', 811),
     ('via', 1783),
     ('video', 604),
     ('vs', 555),
     ('w/', 729),
     ('waktu', 544),
     ('want', 1024),
     ('way', 724),
     ('week', 792),
     ('well', 518),
     ('without', 2839),
     ('work', 716),
     ('worse', 914),
     ('would', 1009),
     ('x', 5906),
     ('yet', 661)]

## Users

Pain >30

    [('acute', 37),
     ('back', 489),
     ('chronic', 1610),
     ('feel', 33),
     ('foot', 31),
     ('interventional', 76),
     ('joint', 102),
     ('knee', 39),
     ('less', 31),
     ('muscle', 31),
     ('natural', 49),
     ('neck', 150),
     ('neuropathic', 31),
     ('pain', 86),
     ('pediatric', 38),
     ('pelvic', 74),
     ('persistent', 30),
     ('reduce', 47),
     ('regional', 76),
     ('relieve', 63),
     ('without', 39)]

## Tweets

Addict (>30)

    [('beauty', 48),
     ('blogger', 44),
     ('book', 138),
     ('caffeine', 116),
     ('cat', 42),
     ('chocolate', 83),
     ('coffee', 690),
     ('disney', 42),
     ('enjoy', 32),
     ('fashion', 66),
     ('fitness', 88),
     ('food', 98),
     ('football', 40),
     ('gym', 105),
     ('instagram', 38),
     ('internet', 62),
     ('love', 106),
     ('lover', 97),
     ('makeup', 61),
     ('media', 129),
     ('mom', 30),
     ('movie', 98),
     ('music', 340),
     ('netflix', 35),
     ('new', 41),
     ('news', 65),
     ('polish', 31),
     ('porn', 31),
     ('recovering', 35),
     ('series', 42),
     ('sex', 42),
     ('shoe', 52),
     ('show', 30),
     ('sports', 77),
     ('starbucks', 76),
     ('tattoo', 65),
     ('tea', 182),
     ('travel', 80),
     ('tv', 200),
     ('twitter', 79),
     ('visit', 62),
     ('wife', 39),
     ('writer', 58),
     ('youtube', 44)]

## User profiles

Addict (>30)

    [('beauty', 48),
     ('blogger', 44),
     ('book', 138),
     ('caffeine', 116),
     ('cat', 42),
     ('chocolate', 83),
     ('coffee', 690),
     ('disney', 42),
     ('enjoy', 32),
     ('fashion', 66),
     ('fitness', 88),
     ('food', 98),
     ('football', 40),
     ('gym', 105),
     ('instagram', 38),
     ('internet', 62),
     ('love', 106),
     ('lover', 97),
     ('makeup', 61),
     ('media', 129),
     ('movie', 98),
     ('music', 340),
     ('netflix', 35),
     ('new', 41),
     ('news', 65),
     ('polish', 31),
     ('porn', 31),
     ('recovering', 35),
     ('series', 42),
     ('sex', 42),
     ('shoe', 52),
     ('sports', 77),
     ('starbucks', 76),
     ('tattoo', 65),
     ('tea', 182),
     ('travel', 80),
     ('tv', 200),
     ('twitter', 79),
     ('visit', 62),
     ('wife', 39),
     ('writer', 58),
     ('youtube', 44)]

Addiction (>10)
[('alcohol', 23),
 ('coffee', 11),
 ('drug', 28),
 ('new', 11),
 ('sex', 11),
 ('slight', 14),
 ('unhealthy', 14)]

Pain >10
[('acute', 37),
 ('advanced', 15),
 ('alleviate', 14),
 ('alleviating', 10),
 ('ankle', 18),
 ('arthritis', 21),
 ('back', 489),
 ('based', 11),
 ('best', 14),
 ('body', 14),
 ('cancer', 12),
 ('certified', 17),
 ('chronic', 1610),
 ('comprehensive', 14),
 ('constant', 20),
 ('drug-free', 11),
 ('du', 12),
 ('effective', 13),
 ('eliminate', 15),
 ('emotional', 13),
 ('every', 11),
 ('experience', 12),
 ('facial', 21),
 ('family', 11),
 ('feel', 33),
 ('fibromyalgia', 15),
 ('foot', 31),
 ('free', 20),
 ('general', 12),
 ('get', 10),
 ('health', 18),
 ('heel', 22),
 ('help', 17),
 ('hip', 10),
 ('holistic', 10),
 ('https', 18),
 ('interventional', 76),
 ('intractable', 17),
 ('joint', 102),
 ('knee', 39),
 ('know', 22),
 ('le', 12),
 ('less', 31),
 ('let', 14),
 ('life', 14),
 ('limb', 11),
 ('live', 21),
 ('living', 19),
 ('love', 23),
 ('manage', 12),
 ('medical', 10),
 ('mom', 16),
 ('much', 19),
 ('multidisciplinary', 10),
 ('muscle', 31),
 ('musculoskeletal', 22),
 ('natural', 49),
 ('neck', 150),
 ('nerve', 21),
 ('neuropathic', 31),
 ('new', 10),
 ('pain', 86),
 ('pediatric', 38),
 ('pelvic', 74),
 ('persistent', 30),
 ('physical', 27),
 ('providing', 17),
 ('real', 11),
 ('reduce', 47),
 ('reducing', 12),
 ('regional', 76),
 ('relieve', 63),
 ('relieves', 11),
 ('relieving', 17),
 ('responsible', 11),
 ('severe', 15),
 ('shoulder', 29),
 ('specialist', 14),
 ('spinal', 15),
 ('spine', 16),
 ('sports', 11),
 ('stop', 13),
 ('therapy', 10),
 ('time', 13),
 ('topical', 11),
 ('treating', 25),
 ('true', 11),
 ('us', 20),
 ('w/chronic', 10),
 ('wants', 19),
 ('without', 39)]

# Create stopword free db

In [2]:
%cd twitteranalysis
import sqlite3
from nltk.corpus import stopwords
import environment

(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis


In [ ]:
def remove_old_table_name(db):
    q = "alter table word_map_deux rename to word_map"
    conn = sqlite3.connect(db)
    conn.execute(q)
    conn.commit()
    conn.close()

In [3]:
english_stops = stopwords.words('english')

def make_stopword_exclusion_query(stops):
    st = " AND word != '%s'"
    query = "SELECT * FROM word_map WHERE word != '2' "
    for s in stops:
        n = st % s
        query += n 
    return query


def row_generator(source_db, query):
    conn = sqlite3.connect(source_db)
    try:
        r = conn.execute(query)
        while True:
            v = r.fetchone()
            if v is None: raise StopIteration
            yield v
    finally:
        conn.close()


query = make_stopword_exclusion_query(english_stops)


In [4]:
def remove_stopwords(source_db, target_db):
    query = make_stopword_exclusion_query(english_stops)

    conn2 = sqlite3.connect(target_db)

    try:
        userQuery = """INSERT INTO word_map (tweet_id, user_id, word, sentence_index, word_index) VALUES (?, ?, ?, ?, ?)"""
        conn2.executemany( userQuery, row_generator(source_db, query) )
        conn2.commit()
    except StopIteration:
        print('stopped')
    finally:
        conn2.close()
        print('done')



In [6]:

gen = row_generator(environment.TWEET_DB_MASTER, query)
conn2 = sqlite3.connect(environment.TWEET_DB_NO_STOP)

size = 1000
queue = []

cnt = 0


def save_rows(rows, conn):
#     print('save_rows called; count is %s' % cnt)
    userQuery = """INSERT INTO word_map (tweet_id, user_id, word, sentence_index, word_index) VALUES (?, ?, ?, ?, ?)"""
    conn.executemany( userQuery, rows )
    conn.commit()


prev = 'j'
try:

    while True:    
        cnt += 1
        try:
            if len(queue) >= size:
                save_rows(queue, conn2)
                queue = []
            v = next(gen)
            prev = v
            if v is not None:
                queue.append(v)

        except StopIteration:
            save_rows(queue, conn2)
            
            break

    print('jip')
    
except Exception as e:
    print(e)
    print(prev)
    
finally:
    conn2.close()
    print("count is %s" % cnt)


jip
count is 59756273


/Users/adam/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'row_generator' raised StopIteration


In [ ]:
15510416 - n[0]